In [175]:
import numpy as np
import time
import math
import pandas as pd
import itertools
import requests
from bs4 import BeautifulSoup
import sympy
import xlwings as xw
import copy
from sympy.abc import a,b,c,d,e,f,g,h,i,j,k
from scipy.special import binom

In [176]:
print("Puzzle")
print("~~~~~~")
url='https://www.janestreet.com/puzzles/figurine-figuring/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
x =[text for text in soup.body.stripped_strings]
print("".join(x[7:]))

from sympy.abc import a,b,c,d,e,f,g,h,i,j,k


Puzzle
~~~~~~
Jane received 78 figurines as gifts this holiday season:  12 drummers drumming, 11 pipers piping, 10 lords a-leaping, etc., down to 1 partridge in a pear tree.   They are all mixed together in a big bag.  She agrees with her friend Alex that this seems like too many figurines for one person to have, so she decides to give some of her figurines to Alex.   Jane will uniformly randomly pull figurines out of the bag one at a time until she pulls out the partridge in a pear tree, and will give Alex all of the figurines she pulled out of the bag (except the partridge, that’s Jane’s favorite).Ifnis the maximum number of any one type of ornament that Alex gets, what is the expected value ofn, to seven significant figures?


In [177]:
# Generating function to work out the number of combinations for the various pulls.
# number of ways of making 

terms = [ (1+a),
          (1+a+a**2),
          (1+a+a**2+a**3),
          (1+a+a**2+a**3+a**4),
          (1+a+a**2+a**3+a**4+a**5),
          (1+a+a**2+a**3+a**4+a**5+a**6),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7+a**8),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7+a**8+a**9),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7+a**8+a**9+a**10),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7+a**8+a**9+a**10+a**11),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7+a**8+a**9+a**10+a**11+a**12),
          ]
    
    
perms=np.zeros((12,78))

perms[0,78-12:] =(sympy.Poly(terms[0]**11).coeffs())

for i in range(1,12):
    p = sympy.Poly(1,a)
    
    for j in range(1,12):
        p *= sympy.Poly(terms[min(i,j)])
    perms[i,78-len(p.coeffs()):]=p.coeffs()    

In [178]:
processed = np.zeros((12,78),dtype=int)
processed[0,:]=perms[0,:]
for i in range(11,0,-1):
    processed[i,:] =perms[i,:] - perms [i-1,:]

processed = processed/np.sum(processed,axis=0)*np.array([np.arange(1,13)]).T

print("Expectation is : {:.7f}".format(np.sum(processed)/78))


Expectation is : 8.0799297


In [179]:
# Check with a simulation

urn = np.array(
      [1,
       2,2,
       3,3,3,
       4,4,4,4,
       5,5,5,5,5,
       6,6,6,6,6,6,
       7,7,7,7,7,7,7,
       8,8,8,8,8,8,8,8,
       9,9,9,9,9,9,9,9,9,
       10,10,10,10,10,10,10,10,10,10,
       11,11,11,11,11,11,11,11,11,11,11,
       12,12,12,12,12,12,12,12,12,12,12,12],dtype=int)

def loop():
    temp = copy.copy(urn)
    np.random.shuffle(temp)
    picks = np.zeros(11)
    for draw in temp:
        if draw == 1:
            return np.max(picks)
        else:
            picks[draw-2] +=1
            
start = time.perf_counter()
sum = 0
for i in range(100000):
    sum += loop()
    if i % 10000 ==0:
        print(sum/(i+1))
        
stop =  time.perf_counter()
print('\n Solution took {:0.4f} seconds\n'.format((stop-start)))

9.0
6.884611538846116
6.887405629718514
6.886970434318856
6.887102822429439
6.884502309953801
6.873635439409343
6.866101912829817
6.862214222322221
6.865412606526594

 Solution took 4.3701 seconds



In [180]:
#expectation for 2 pulls longhand

grid = np.zeros((11,11))

for i,j in itertools.product(range(11),range(11)):
    grid[i,j] = (i+2)/77*(j+2-(i==j))/76
    
print("expectation for 2 pulls = ",1+np.sum(grid * np.eye(11)))

print("from above calc is : ",np.sum(processed[:,75]))

expectation for 2 pulls =  1.0977443609022557
from above calc is :  1.1666666666666667


In [142]:
terms = [
          (1+a+a**2),
          (1+b+b**2+b**3),
          (1+c+c**2+c**3+c**4),
          (1+d+d**2+d**3+d**4+d**5),
          (1+e+e**2+e**3+e**4+e**5+e**6),
          (1+f+f**2+f**3+f**4+f**5+f**6+f**7),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7+a**8),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7+a**8+a**9),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7+a**8+a**9+a**10),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7+a**8+a**9+a**10+a**11),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7+a**8+a**9+a**10+a**11+a**12),
          ]
    